# Packaging

In this lesson we'll learn about
* Final package additions
    * `README.md`
    * `__init__.py`
    * `setup.py`
* Setting up your pypirc file
* Wheels and eggs
* Uploading packages 
    * To the pypi test server
    * To the pypi real server
    * To our internal artifactory servers
* Further reading
    * pyproject.toml
    * flit
    * poetry

Following on from last time, we should now have a project that looks something like this

```bash
my_project
├── my_project
│   ├── __init__.py
│   └── my_module.py
└── setup.py
```

Now we have our layout, we need a way to tell python how to install the package. This is done using the setup.py. Here's the simplest we could use to install the above package:

```python
from setuptools import setup

setup(name='my_project',
      packages=['my_project'])
```

Although that will work, you probably want to add a lot more information than that. We'll go into detail later.

There are many other things that can be included, but this is a good start. One thing I haven't included here is the license. Generally it's a good idea to include a license if you're going to publish the package in a public place. https://choosealicense.com/ is a good resource for this. 

## `README.md`

This isn't a hard requirement, but it's good practice to add. 

The md suffix stands for MarkDown, so that's the format to write it in. 

If you add this in the root of the git repo, then it will be visible on the main page of GitHub, Stash (BitBucket) or wherever you're hosting it.

## `__init__.py`

You'll notice in the package above there's a `__init__.py` file. Strictly speaking they're no long required to build a package, but it does allow us to do a few quite useful things. 

As you remember from last time, the `__init__.py` is run when we import a package. If we had a package structure similar to the one above and we wanted to import a function from it called `say_hello`. We would have to do

```python
>>> from my_project.my_module import say_hello
>>> say_hello()
Hello
```

To make things a bit less cumbersome, we can import the function in the `__init__.py` file. 

```python
from my_module import *
```

Then we can import directly from the package

```python
>>> from my_project import say_hello
>>> say_hello()
Hello
```

You could of course be specific in the `__init__.py` and just import the objects you want. Normally an import * would be frowned upon, but it's fine here as you're not polluting any other namespaces.

## `setup.py`

```python
import setuptools

with open("README.md", "r") as f:
    long_description = f.read()

setuptools.setup(
    name="unique-package-name",
    version="0.0.1",
    author="Example Author",
    author_email="author@example.com",
    description="A small example package",
    long_description=long_description,
    long_description_content_type="text/markdown",
    url="https://hq-stash.corp.proofpoint.com/users/mrobinson/repos/example-package/browse",
    packages=setuptools.find_packages(),
    classifiers=[
        "Programming Language :: Python :: 3",
        "Operating System :: OS Independent",
    ],
)
```

This one has a lot more information and is a lot more useful. 

One of the nice parts of this one is the reading in of the `README.md` and using it as the long description. 

You'll also see a list of classifiers, these are useful when searching for packages. You can get a full list of them at https://pypi.org/classifiers/

Now you should be able to install it to your system with

```bash
pip install .
```

or as I mentioned in the last lesson, you can install it in editable mode. (creates symlinks so you can still edit the installed version)

```bash
pip install -e .
```

Make sure that you can complete this step before moving on any further. 

## `.pypirc` file

Before we set up our file, we need to do a few things to get ready. 

Let's install the tools we're going to need to get builds and be able to upload. You could install these in a virtual env, but we'll be using them across multiple environments and probably won't be editing them, so it's fine to install to the system. 

First the tools to deal with and create packages

```bash
pip3 install -U setuptools wheel --upgrade
```

Next we need twine, which is the tool for uploading the newly created packages

```bash
pip3 install -U twine --upgrade
```

The last thing to do before moving on is create a new user account on https://test.pypi.org/account/register/

There are two main benefits to using a .pypirc file: 

1. It removes the need to enter a username/password when pushing to PyPI. 
2. It simplifies command line usage when pushing packages to a non-default package repository (e.g. artifactory or the pypi test repo). 

The official documentation on the .pypirc file can be found at https://docs.python.org/3/distutils/packageindex.html#the-pypirc-file.

```
[distutils]
index-servers =
    dev-pypi-local
    pypi
    testpypi

[dev-pypi-local]
repository: https://repocache.nonprod.ppops.net/artifactory/api/pypi/dev-pypi-local
username: <YOURUSERNAME>

[pypi]
username: <YOURUSERNAME>
password: 1234567890abcde

[testpypi]
repository: https://test.pypi.org/legacy/
username: <YOURUSERNAME>
password: 1234567890abcde
```

The password field is optional, if you don't add it then you will be prompted for it. 

Also note that the pypi test and official repos do not use the same login credentials, so you will need to register for both if you wish to upload to both. 
During this lesson we will only be uploading to the test repo.

## Wheels and eggs

Long story short, these are both distribution formats but wheel is considered the standard now. 

They are both packaging formats which aim to give an artifact that doesn't need to be built or compiled. 

So now that we have all the pieces in place, we can build the wheel.

```bash
python3 setup.py sdist bdist_wheel
```

You should see that this then creates two files in your `dist` directory. The wheel file, and the source distribution (the tgz file). 

These are the files that we will use to upload to the repos. 

## Uploading to test pypi repos

Since we set up our pypirc file earlier, this should be quite simple

```bash
twine upload --repository testpypi dist/* 
```

The argument given to the `--repository` switch is a reference to our config file. So if you added your user name and password there, it should just go ahead and upload it for you right away. 

You can use the full URL of the repo if you happen to be on a box where you don't have this file

```bash
twine upload --repository-url https://test.pypi.org/legacy/ dist/*
```

However this will always ask for a username and password, that can be overridden with the -u and -p flags. Though the first method is preferred if you wish to add this to an automation set up. 

Be sure to set the correct permissions on your .pypirc file (`chmod 600 ~/.pypirc`) if you decide to do this, so as to protect your password. 

## Testing it worked

Basically, we point it at the correct repo, using the `--index-url` switch and tell it not to grab any dependencies (`--no-deps`). 

If you've set up your `.pypirc` file correctly, you shouldn't actually need to change the index as it will search pypi test, and if you don't have any dependenices you don't need to tell. However, this is good practice and this is the command you will notmall use. 

```bash
pip install --index-url https://test.pypi.org/simple/ --no-deps <YOURPACKAGENAME>
```

## Uploading to the real pypi repos

The process is much the same as before, but you would use the other repo defined in your `.pypirc` file. 

```bash
twine upload --repository pypi dist/* 
```

There are a few other things to note however
1. The package name you choose must be unique. You can check the site or do a `pip search` before you choose.
2. You'll need a new account for the real servers, logins are not shared with the test servers.
3. You should just install the package as you would normally, i.e. `pip install <PackageName>`

## Uploading to artifactory

The steps are the same as the others, just change the repository

```bash
twine upload --repository dev-pypi-local dist/* 
```

You will however need to raise a SYSOPS ticket to get access to push to the server. 

If your team has a Jenkins server, you may already have a job or credentials that can be used to push. 

## Further reading

Once you understand what's going on with the above tools and how it all fits together, there are a number of other ways to do it that *can* simplify things for complicated projects and workflows. 

Both Flit and Poetry use a file called pyproject.toml which aims to move away from the setup.py format for various reason which you're better looking up in your own time!

### Flit
* https://flit.readthedocs.io/en/latest/
* https://github.com/takluyver/flit

### Poetry
* https://poetry.eustace.io/docs/
* https://github.com/sdispater/poetry

### realpython.org

This is a really great site, there are a tonne of articles worth reading on it. The packaging one will cover a lot of the things I've touched on here in more depth.

* https://realpython.com/pypi-publish-python-package/